# AIPI 590 - XAI | Assignment #04

### Osama Ahmed


![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=green)
[![View notebooks on Github](https://img.shields.io/static/v1.svg?logo=github&label=Repo&message=View%20On%20Github&color=lightgrey)](https://github.com/osama-shawir/Interpretable-ML-Models/blob/main/interpretable_ml.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osama-shawir/Interpretable-ML-Models/blob/main/interpretable_ml.ipynb)  

This project involves the implementation of three interpretable machine learning models from the imodels library. The models are:

1. RuleFit
2. OneR
3. Greedy Rule Tree

In [5]:
# Import necessary libraries
# Upgrade scikit-learn to the latest version
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from imodels import RuleFitClassifier, OneRClassifier, GreedyRuleTreeClassifier

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Initialize the models
rulefit = RuleFitClassifier()
oner = OneRClassifier()
greedy_rule_tree = GreedyRuleTreeClassifier()

# Train the models
rulefit.fit(X_train, y_train)
oner.fit(X_train, y_train)
greedy_rule_tree.fit(X_train, y_train)

# Make predictions
y_pred_rulefit = rulefit.predict(X_test)
y_pred_oner = oner.predict(X_test)
y_pred_greedy_rule_tree = greedy_rule_tree.predict(X_test)

# Plot confusion matrices
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for ax, model_name, y_pred in zip(
    axes,
    ["RuleFit", "OneR", "Greedy Rule Tree"],
    [y_pred_rulefit, y_pred_oner, y_pred_greedy_rule_tree],
):
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=iris.target_names)
    disp.plot(ax=ax, cmap=plt.cm.Blues)
    ax.set_title(f"{model_name} Confusion Matrix")

plt.tight_layout()
plt.show()

     ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.5 MB 682.7 kB/s eta 0:00:11
     ---------------------------------------- 0.1/7.5 MB 787.7 kB/s eta 0:00:10
     -- ------------------------------------- 0.4/7.5 MB 3.1 MB/s eta 0:00:03
     ---- ----------------------------------- 0.8/7.5 MB 4.5 MB/s eta 0:00:02
     ------ --------------------------------- 1.2/7.5 MB 5.9 MB/s eta 0:00:02
     -------- ------------------------------- 1.5/7.5 MB 6.1 MB/s eta 0:00:01
     ---------- ----------------------------- 2.0/7.5 MB 6.6 MB/s eta 0:00:01
     ------------- -------------------------- 2.5/7.5 MB 6.8 MB/s eta 0:00:01
     ---------------- ----------------------- 3.0/7.5 MB 7.7 MB/s eta 0:00:01
     ----------------- ---------------------- 3.3/7.5 MB 7.6 MB/s eta 0:00:01
     -------------------- ------------------- 3.8/7.5 MB 7.9 MB/s eta 0:00:01
     ----------------------- ---------------- 4.5/7.5 MB 8.4 MB/s e

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [36 lines of output]
      <string>:17: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
      Partial import of sklearn during the build process.
      <string>:116: DeprecationWarning:
      
        `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
        of the deprecation of `distutils` itself. It will be removed for
        Python >= 3.12. For older Python versions it will remain present.
        It is recommended to use `setuptools < 60.0` for those Python versions.
        For more details, see:
          https://numpy.org/devdocs/reference/distutils_status_migration.html
      
      
      Traceback (most recent call last):
        File "C:\Users\osama\miniconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      

ImportError: cannot import name 'indices_to_mask' from 'sklearn.utils._mask' (c:\Users\osama\miniconda3\Lib\site-packages\sklearn\utils\_mask.py)

In [12]:
!pip install --upgrade imodels
!pip install pandas scikit-learn matplotlib seaborn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree, DecisionTreeClassifier
from sklearn import metrics

# installable with: `pip install imodels`
from imodels import OneRClassifier
# from imodels import SLIMRegressor, BayesianRuleListClassifier, RuleFitRegressor, GreedyRuleListClassifier
# from imodels.discretization import ExtraBasicDiscretizer
import demo_helper
np.random.seed(13)

ImportError: cannot import name 'indices_to_mask' from 'sklearn.utils._mask' (c:\Users\osama\miniconda3\Lib\site-packages\sklearn\utils\_mask.py)